In [1]:
from bs4 import BeautifulSoup
import requests
import time
import pymongo
from splinter import Browser
import pandas as pd
from selenium import webdriver

In [2]:
# NASA Mars News
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [3]:
html = requests.get(url)

In [4]:
soup = BeautifulSoup(html.text, 'html.parser')

In [5]:
news_title = soup.find("div", class_="content_title").findChildren()[0].string.strip()
news_p = soup.find("div", class_="rollover_description_inner").text.strip()

In [6]:
# JPL Mars Space Images - Featured Image

In [7]:
browser = webdriver.Chrome()

In [8]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.get(url)
time.sleep(3)

In [9]:
browser.find_element_by_link_text('FULL IMAGE').click()

In [10]:
time.sleep(2)
browser.find_element_by_link_text('more info').click()
time.sleep(5)

In [11]:
soup = BeautifulSoup(browser.page_source, "html.parser")

In [12]:
featured_image_url = "https://www.jpl.nasa.gov" + soup.find("img", class_="main_image")['src']
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19673_hires.jpg'

In [ ]:
# Mars Weather

In [ ]:
url = "https://twitter.com/marswxreport?lang=en"
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')

In [ ]:
mars_weather = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text

In [ ]:
url = "https://space-facts.com/mars/"
html = requests.get(url)
soup = BeautifulSoup(html.text, 'html.parser')
table = soup.find("table", id="tablepress-mars")

In [ ]:
table_string = str(table)

In [ ]:
data = pd.read_html(table_string)
data = data[0]

In [ ]:
data = dict(zip(data[0].tolist(), data[1].tolist()))

In [ ]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.get(url)

In [ ]:
hemisphere_list = ["Cerberus Hemisphere Enhanced", "Schiaparelli Hemisphere Enhanced", "Syrtis Major Hemisphere Enhanced", "Valles Marineris Hemisphere Enhanced"]
img_url_list = []
for hemisphere in hemisphere_list:
    browser.get(url)
    browser.find_element_by_link_text(f"{hemisphere}").click()
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    img_url = soup.find_all("a", text="Sample")[0]['href']
    img_url_list.append(img_url)

In [ ]:
hemisphere_image_urls = [
    {"title": "Cerberus Hemisphere", "img_url": "..."},
    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
]

In [ ]:
for dictionary in hemisphere_image_urls:
        dictionary['img_url']

In [ ]:
for x in range(len(hemisphere_image_urls)):
    hemisphere_image_urls[x]['img_url'] = img_url_list[x]

In [ ]:
def scrape():
    scraped_data = {
        "News Title": news_title,
        "News Paragraph": news_p,
        "Featured Image URL": featured_image_url,
        "Mars Weather": mars_weather,
        "Mars Data": data,
        "Hemisphere Image Urls": hemisphere_image_urls }
    return scraped_data

In [ ]:
scraped_data = scrape()

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
mars = client.marsDB

In [ ]:
results = mars.mission.find()

In [ ]:
results_list = []
for row in results:
    results_list.append(row)
results_list

In [ ]:
try:
    item = mars.mission.find_one()
    item_id = item.get('_id')
    mars.mission.update_one({'_id': item_id}, {"$set": scraped_data}, upsert=True)
except AttributeError: 
    mars.mission.insert_one(scraped_data)

In [ ]:
item_id

In [ ]:
scraped_data

In [ ]:
results = mars.mission.find()
results_list = []
for row in results:
    results_list.append(row)
len(results_list)

In [ ]:
results = mars.mission.find()
results_list = []
for row in results:
    results_list.append(row)
results_list